In [1]:
import pandas as pd
import glob
import geopandas as gpd

In [65]:
raw_countries=gpd.read_file('fao/shp_from_un/BNDA25_CTY.shp')

In [2]:
crops_list=glob.glob('crop_model_from_fao_v2/*csv')
crops_list=sorted(crops_list)
len(crops_list)

27

In [133]:
polination_list=glob.glob('polinatior_model_from_fao_v2/*csv')
polination_list=sorted(polination_list)
len(polination_list)

27

In [134]:
price_list=glob.glob('fao_price/*csv')
price_list=sorted(price_list)
len(price_list)

28

In [135]:
#drop 1991 data
price_list=price_list[1:]

In [137]:
for index,p in enumerate(price_list):
    # read 3 target csv
    price_df=pd.read_csv(p)
    crop_df=pd.read_csv(crops_list[index])
    polination_df=pd.read_csv(polination_list[index])
    #first we merge crop and pollination csv
    m=pd.merge(crop_df,polination_df,how='left',right_on='Country',left_on='Country')
    #drop columns
    m.drop(['Split_Year_x', 'What_happened_x',
            'Country_split_(by_area_code)_x', 'Percentages_x','Area_y', 'Area Code_y', 'Split_Year_y',
            'What_happened_y', 'Country_split_(by_area_code)_y', 'Percentages_y','Year_y'],axis=1,inplace=True)
    #merge price csv with first merged csv
    m2=pd.merge(m,price_df,how='left',right_on='Country',left_on='Country')
    #drop columns
    m2.drop(['Area', 'Area Code', 'Split_Year', 'What_happened',
             'Country_split_(by_area_code)', 'Percentages', 'Domain Code', 'Domain',
             'Element Code', 'Element'],axis=1,inplace=True)
    # name fixing
    m2.rename(columns={'Area_x':'Area'},inplace=True)
    m2.rename(columns={'Area Code_x':'Area_Code'},inplace=True)
    m2.rename(columns={'Year_x':'Year'},inplace=True)
    # merge with shp UN shp file
    den=pd.merge(raw_countries,m2,how='left', left_on='ISO3CD', right_on='Country')
    year_info=p.split('_')[-1].split('.')[0]
    #export data as a shp file
    den.to_file(f'fao_merge_crop_poll_price/fao_merged_{year_info}.shp',driver='ESRI Shapefile')